In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
MODEL = "llama2"

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Sure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope that brought a smile to your face!"

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
chain = prompt | model | parser

chain.input_schema.schema()

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Sure! Based on the context you provided, my answer to the question "What\'s your name?" would be: Santiago.'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Hepatitis_A_Dose2_Doc.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='U.S. Department of \nHealth and Human Service s \nCenters for Disease Control and Prevention VACCINE INFORMATION STATEMENT\nHepatitis A Vaccine:\nWhat You Need to KnowMany vaccine information statements are \navailable in Spanish and other languages.  \nSee www.immunize.org/vis\nHojas de información sobre vacunas están disponibles en español y en muchos otros idiomas. Visite www.immunize.org/vis\n1. Why get vaccinated?\nHepatitis A vaccine can prevent hepatitis A .\nHepatitis A is a serious liver disease. It is usually \nspread through close, personal contact with an \ninfected person or when a person unknowingly \ningests the virus from objects, food, or drinks that \nare contaminated by small amounts of stool (poop) \nfrom an infected person.\nMost adults with hepatitis A have symptoms, \nincluding fatigue, low appetite, stomach pain, \nnausea, and jaundice (yellow skin or eyes, dark urine, \nlight-colored bowel movements). Most children less \nthan 6 years of

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [8]:
retriever = vectorstore.as_retriever()
retriever.invoke("Hepatitis")

[Document(page_content='OFFICE  \nUSE  ONLY3.  T alk with your health  \ncare provider\nTell your vaccination provider if the person getting \nthe vaccine:\n \x82Has had an allergic reaction after a previous dose \nof hepatitis A vaccine, or has any severe, life-\nthreatening allergies\nIn some cases, your health care provider may  decide to postpone hepatitis A vaccination until  \na future visit.\nPregnant or breastfeeding people should be \nvaccinated if they are at risk for getting hepatitis\xa0A. \nPregnancy or breastfeeding are not reasons to avoid \nhepatitis A vaccination.\nPeople with minor illnesses, such as a cold, may be \nvaccinated. People who are moderately or severely ill should usually wait until they recover before getting \nhepatitis\xa0A vaccine.\nY our health care provider can give you more \ninformation.\n4. Risks of a vaccine reaction\n \x82Soreness or redness where the shot is given, fever, \nheadache, tiredness, or loss of appetite can happen \nafter hepatitis 

In [9]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [10]:
questions = [
    "What is Hepatitis A?",
    "How many doses of Hepatitis A vaccine the children needs?",
    "Whom to contact or email if there is a serious problem?",
    "What are the risks of a vaccine reaction?",
    "How much does the vaccine cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is Hepatitis A?
Answer: Based on the provided document, Hepatitis A is a serious liver disease that is usually spread through close, personal contact with an infected person or when a person unknowingly ingests the virus from objects, food, or drinks that are contaminated by small amounts of stool from an infected person.

Question: How many doses of Hepatitis A vaccine the children needs?
Answer: Based on the provided document, children need two doses of Hepatitis A vaccine:

* The first dose should be given at 12 through 23 months of age.
* The second dose should be given at least 6 months after the first dose.

So, the answer to your question is two doses of Hepatitis A vaccine for children.

Question: Whom to contact or email if there is a serious problem?
Answer: Based on the information provided in the document, if there is a serious problem regarding hepatitis A vaccination, individuals can contact their healthcare provider or local health department. They can als